In [ ]:
!pip install nagisa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 48.8 MB/s eta 0:00:00


# 顔文字の抽出

In [ ]:
KAOMOJI_MIN = 4
KAOMOJI_MAX = 15

In [ ]:
import nagisa
import unicodedata


def extract_kaomoji(text):
  words = nagisa.extract(text, extract_postags=['補助記号']).words

  if len(words) == 0:
    return None

  max_len = len(max(words, key=len))
  if max_len < KAOMOJI_MIN:
    return None

  while words[-2:] == ['[', ']']:
    words = words[:-2]

  result = ''.join(words)
  if len(result) > KAOMOJI_MAX:
    return None

  return result

In [ ]:
in_files = ['simeji', 'kmoji', 'kaomoji_copy']

In [ ]:
for name in in_files:
  with open(name + '_extracted.txt', mode='w', encoding='utf-8') as out_file:
    with open(name + '.txt', mode='r') as in_file:
      for line in in_file:
        result = extract_kaomoji(line)
        if result != None:
          out_file.write(result + '\n')

# ファイルをまとめる

In [ ]:
kaomoji_data = []

for name in in_files:
  with open(name + '_extracted.txt', mode='r') as in_file:
    for line in in_file:
      kaomoji_data.append(line)

In [ ]:
with open('kaomoji_MAX=' + str(KAOMOJI_MAX) + '.txt', \
           mode='w', encoding='utf-8') as out_file:
  for data in list(set(kaomoji_data)):
    out_file.write(data)

# 顔文字を反転させてデータ拡張

In [35]:
pairs = [('(',     ')'),
         ('　́',   '`'),
        #  ('′',    '`'),ヽ
        #  ('\'',    '`'),
         ('ノ',    'ヾ'),
         ('/',     '\\'),
         ('>',     '<'),
         ('≧',    '≦')]

In [36]:
import numpy as np

def kaomoji_reverse(kmj):
  kmj_rv = np.array(list(reversed(kmj)))

  for pair in pairs:
    mask0 = kmj_rv == pair[0]
    mask1 = kmj_rv == pair[1]
    kmj_rv[mask0] = pair[1]
    kmj_rv[mask1] = pair[0]

  return kmj_rv

In [55]:
KAOMOJI_MAX = 20
kmj_list = []

with open('kaomoji_MAX=' + str(KAOMOJI_MAX) + '.txt', mode='r') as file:
  for line in file:
    kmj = list(line.replace('\n', ''))
    kmj_rv = kaomoji_reverse(kmj)
    kmj_list.append(''.join(kmj))
    kmj_list.append(''.join(kmj_rv))

In [56]:
len(kmj_list)

23174

In [57]:
KAOMOJI_MAX = 20
with open('kaomoji_MAX=' + str(KAOMOJI_MAX) + '_DA.txt', \
           mode='w', encoding='utf-8') as out_file:
  for kmj in list(set(kmj_list)):
    if len(kmj) > KAOMOJI_MAX:
      continue
    out_file.write(kmj + '\n')